In [2]:
# coding: utf-8
import numpy as np
import pandas as pd
# 全部行都能输出

In [44]:
table_target = pd.read_csv(
    "../data/data/trainingset/orderFuture_train.csv", sep=",", encoding="UTF-8")  # 读取目标属性表
user_info = pd.read_csv("../data/data/trainingset/userProfile_train.csv",
                        sep=",", encoding="UTF-8")  # 读取用户基本属性表
order_info = pd.read_csv(
    "../data/data/trainingset/orderHistory_train.csv", sep=",", encoding="UTF-8")  # 读取历史订单数据表

In [23]:
# 没有使用action_info
# user_com = pd.read_csv("../data/data/trainingset/userComment_train.csv",sep=',',encoding="UTF-8")


# user_info = pd.read_csv("../data/data/test/userProfile_test.csv",
#                         sep=",", encoding="UTF-8")  # 读取用户基本属性表
# order_info = pd.read_csv(
#     "../data/data/test/orderHistory_test.csv", sep=",", encoding="UTF-8")  # 读取历史订单数据表

In [45]:
order_info["orderTime"] = pd.to_datetime(order_info["orderTime"], unit="s")

# ## 一、数据探索
# ### 1. 用户基本属性表
# #### 1.1 数据转换
# 原始数据需要经过转换才能符合建模需求，例如：生成新字段、重新分类、变换哑变量、去除重复数据、过滤极端\异常值、填补缺失值、变量聚类、离散化等

# In[47]:

user_info = user_info.fillna("未知")  # 空值填补

In [46]:
order_info.dtypes

userid                int64
orderid               int64
orderTime    datetime64[ns]
orderType             int64
city                 object
country              object
continent            object
dtype: object

In [47]:
# #### 1.2 数据探索
# 除了describe计算基本统计量外，还需要探索目标变量和自变量以及自变量之间的关系，以决定哪些变量入选模型，这种数据探索往往结合图形来说明。

# 常用的绘图包包括matplotlib、seaborn等

user_orderType = user_info.merge(table_target, on="userid", how="left")
# 合并

gender_orderType = user_orderType.groupby("gender", as_index=False)[
    "orderType"].agg({"总数": np.size, "精品订单数": np.sum})
gender_orderType["精品订单率"] = gender_orderType["精品订单数"] / \
    gender_orderType["总数"]
gender_orderType.head()

# In[62]:

province_orderType = user_orderType.groupby("province", as_index=False)[
    "orderType"].agg({"总数": np.size, "精品订单数": np.sum})
province_orderType["精品订单率"] = province_orderType["精品订单数"] / \
    province_orderType["总数"]
province_orderType.sort_values(by="精品订单数", ascending=False)

# In[63]:

age_orderType = user_orderType.groupby("age", as_index=False)[
    "orderType"].agg({"总数": np.size, "精品订单数": np.sum})
age_orderType["精品订单率"] = age_orderType["精品订单数"] / age_orderType["总数"]
age_orderType.sort_values(by="精品订单率", ascending=False)

# ## 筛选变量
# 筛选变量时可以应用专业知识，选取与目标字段相关性较高的字段用于建模，也可通过分析现有数据，用统计量辅助选择
# 为了增强模型稳定性，自变量之间最好相互独立，可运用统计方法选择要排除的变量或进行变量聚类


def type_count(df, type, args):
    count = len(df[df.loc[:, args] == type])
    return count


type1_amount = order_info.groupby("userid").apply(
    type_count, 1, args="orderType")  # 精品订单个数
type0_amount = order_info.groupby("userid").apply(
    type_count, 0, args="orderType")  # 普通订单个数

# In[68]:

type1_amount = pd.DataFrame(type1_amount)
type0_amount = pd.DataFrame(type0_amount)
type1_amount.columns = ["F2.3"]
type0_amount.columns = ["F2.0"]
# 列名

# 订单总数
df1 = type1_amount.merge(type0_amount, on="userid", how="left")
df1["F2.1"] = df1["F2.3"] + df1["F2.0"]

# 删除无用的列
df1 = df1.drop("F2.0", 1)
df1 = df1.drop("F2.3", 1)


def type_count(df, type, args):
    count = len(df[df.loc[:, args] == type])
    return count


type1_exist = order_info.groupby("userid").apply(
    type_count, 1, args="orderType")
type1_exist[type1_exist > 0] = 1
type1_exist = pd.DataFrame(type1_exist)
type1_exist.columns = ["F2.2"]

feature = df1.merge(type1_exist, on="userid", how="left")
feature = feature.merge(type1_amount, on="userid", how="left")

In [108]:
feature.head()

,F2.1,F2.2,F2.3
userid,,,
100000000013,2,0,0
100000000393,1,0,0
100000000459,2,0,0
100000000637,1,0,0
100000000695,1,0,0


In [49]:
# 计算时间间隔
time_gap_max = order_info.groupby("userid")["orderTime"].max()
time_gap_min = order_info.groupby("userid")["orderTime"].min()
time_gap_max = pd.DataFrame(time_gap_max)
time_gap_min = pd.DataFrame(time_gap_min)
time_gap = time_gap_max.merge(time_gap_min, on="userid", how="left")

In [50]:
time_gap.dtypes

orderTime_x    datetime64[ns]
orderTime_y    datetime64[ns]
dtype: object

In [51]:
(time_gap["orderTime_x"] -time_gap["orderTime_y"]) / feature["F2.1"]

userid
100000000013    10122563500000000
100000000393                    0
100000000459      727472500000000
100000000637                    0
100000000695                    0
100000000949      257783333333333
100000000975                    0
100000001023     1117524500000000
100000001231                    0
100000001295                    0
100000001505       87500600000000
100000001955         475000000000
100000002371     1128062750000000
100000003097                    0
100000003461                    0
100000003639     3928866000000000
100000003805                    0
100000004277                    0
100000004297                    0
100000004317                    0
100000005007     3188870000000000
100000006981     1015559666666666
100000008119     1075206500000000
100000008425     8300619000000000
100000008995                    0
100000009017     1731128333333333
100000009463                    0
100000009611      219126000000000
100000009773                    0
1000000

In [ ]:
# feature["F2.17"] = (time_gap["orderTime_x"] -
#                     time_gap["orderTime_y"]) / feature["F2.1"]

In [53]:
f21 = pd.DataFrame(feature.loc[time_gap.index]["F2.1"])
f217 = (time_gap["orderTime_x"] -
        time_gap["orderTime_y"]) / f21["F2.1"]

# f217 = pd.DataFrame(f217)
# f217[0] = f217[0].values / np.timedelta64(1, 'ns')

In [54]:
f217.dtypes

dtype('O')

In [64]:
f217.dtypes==object is bool

False

In [65]:
f2 = f217.dtypes==object
f2

True

In [68]:
isinstance(f2,bool)

True

In [19]:
f217

,0
userid,
100000000371,0.000000e+00
100000001445,2.556372e+06
100000001575,4.315213e+05
100000001737,7.359667e+03
100000001829,0.000000e+00
100000002241,2.038192e+06
100000004161,1.964882e+06
100000004855,0.000000e+00
100000006247,0.000000e+00


In [ ]:
# feature["F2.172"] = f217

In [111]:
temp1 = order_info[order_info["orderType"].isin([1])]
# F2.17订单平均时间间隔
time_gap_max_1 = temp1.groupby("userid")["orderTime"].max()
time_gap_min_1 = temp1.groupby("userid")["orderTime"].min()
time_gap_max_1 = pd.DataFrame(time_gap_max_1)
time_gap_min_1 = pd.DataFrame(time_gap_min_1)
time_gap_1 = time_gap_max_1.merge(time_gap_min_1, on="userid", how="left")

In [112]:
time_gap_1.head()

,orderTime_x,orderTime_y
userid,,
100000001023,2017-01-03 20:19:01,2017-01-03 20:19:01
100000001505,2017-05-18 13:36:06,2017-05-13 18:30:55
100000003461,2017-04-02 10:46:55,2017-04-02 10:46:55
100000005007,2016-12-02 14:45:04,2016-12-02 14:45:04
100000008119,2017-01-09 14:43:40,2017-01-09 14:43:40


In [113]:
time_gap_1.dtypes

orderTime_x    datetime64[ns]
orderTime_y    datetime64[ns]
dtype: object

In [114]:
f23 = pd.DataFrame(feature.loc[time_gap_1.index]["F2.3"])

In [110]:
f23

,F2.3
userid,
100000001023,2
100000001505,4
100000003461,1
100000005007,1
100000008119,1
100000009017,2
100000009773,1
100000009925,2
100000010199,8


In [115]:
(time_gap_1["orderTime_x"] -time_gap_1["orderTime_y"]) / f23['F2.3']

userid
100000001023           0 days 00:00:00
100000001505    1 days 04:46:17.750000
100000003461           0 days 00:00:00
100000005007           0 days 00:00:00
100000008119           0 days 00:00:00
100000009017   30 days 01:18:12.500000
100000009773           0 days 00:00:00
100000009925    0 days 00:41:10.500000
100000010199   27 days 11:07:17.625000
100000014029           0 days 00:00:00
100000015969           0 days 00:00:00
100000016265   45 days 14:50:23.750000
100000017277   36 days 21:12:56.800000
100000017465           0 days 00:00:00
100000019487    0 days 01:19:29.222222
100000021553           0 days 00:00:00
100000022129           0 days 00:02:49
100000022335           0 days 00:00:00
100000022499           0 days 00:00:00
100000022801           0 days 00:15:08
110100016324    0 days 00:12:45.666666
110100016726           0 days 00:00:00
110100033067    0 days 13:06:36.500000
110100116423           0 days 00:00:00
110100273229           0 days 00:22:31
110100512625      

In [116]:
f218 = (time_gap_1["orderTime_x"] -
        time_gap_1["orderTime_y"]) / f23['F2.3']
f218 = pd.DataFrame(f218,columns=['f218'])
# f218[0] = f218[0].values / np.timedelta64(1, 'ns')
feature["F2.18"] = f218

# print(feature.head())

In [118]:
feature.head(16)

,F2.1,F2.2,F2.3,F2.18
userid,,,,
100000000013,2,0,0,NaT
100000000393,1,0,0,NaT
100000000459,2,0,0,NaT
100000000637,1,0,0,NaT
100000000695,1,0,0,NaT
100000000949,3,0,0,NaT
100000000975,1,0,0,NaT
100000001023,6,1,2,0 days 00:00:00
100000001231,1,0,0,NaT


In [119]:
f218.dtypes

f218    timedelta64[ns]
dtype: object

In [19]:
bf218=f218.dtypes==object
bf218

0    False
dtype: bool

In [20]:
(f218.dtypes==object).bool()

False

In [21]:
isinstance(bf218,bool)

False

In [120]:
f218

,f218
userid,
100000001023,0 days 00:00:00
100000001505,1 days 04:46:17.750000
100000003461,0 days 00:00:00
100000005007,0 days 00:00:00
100000008119,0 days 00:00:00
100000009017,30 days 01:18:12.500000
100000009773,0 days 00:00:00
100000009925,0 days 00:41:10.500000
100000010199,27 days 11:07:17.625000


In [100]:
f218.columns

Index(['f218'], dtype='object')

In [77]:
ff218=f218.values

In [ ]:
ff218.

In [78]:
np.unique(ff218)

F:\Anaconda\lib\site-packages\numpy\lib\arraysetops.py:285: FutureWarning: In the future, NAT != NAT will be True rather than False.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


array(['NaT'], dtype='timedelta64[ns]')

In [121]:
f218[0] = f218.values/np.timedelta64(1, 's')

In [124]:
f218[0]

userid
100000001023    0.000000e+00
100000001505    1.035778e+05
100000003461    0.000000e+00
100000005007    0.000000e+00
100000008119    0.000000e+00
100000009017    2.596692e+06
100000009773    0.000000e+00
100000009925    2.470500e+03
100000010199    2.372838e+06
100000014029    0.000000e+00
100000015969    0.000000e+00
100000016265    3.941424e+06
100000017277    3.186777e+06
100000017465    0.000000e+00
100000019487    4.769222e+03
100000021553    0.000000e+00
100000022129    1.690000e+02
100000022335    0.000000e+00
100000022499    0.000000e+00
100000022801    9.080000e+02
110100016324    7.656667e+02
110100016726    0.000000e+00
110100033067    4.719650e+04
110100116423    0.000000e+00
110100273229    1.351000e+03
110100512625    0.000000e+00
110100633862    2.206467e+04
110100818564    8.991900e+04
110100818565    0.000000e+00
110101273620    5.379350e+04
                    ...     
114829653346    0.000000e+00
114829750941    0.000000e+00
114831269641    0.000000e+00
1148320

In [123]:
feature.shape

(10637, 4)

In [125]:
feature['f218']=f218[0]
# NaT是在这个过程中，由于f218的id比较少，缺少的id对应的为nan，然后由nan转成了NaT

In [127]:
feature.head(16)

,F2.1,F2.2,F2.3,F2.18,f218
userid,,,,,
100000000013,2,0,0,NaT,NaN
100000000393,1,0,0,NaT,NaN
100000000459,2,0,0,NaT,NaN
100000000637,1,0,0,NaT,NaN
100000000695,1,0,0,NaT,NaN
100000000949,3,0,0,NaT,NaN
100000000975,1,0,0,NaT,NaN
100000001023,6,1,2,0 days 00:00:00,0.00
100000001231,1,0,0,NaT,NaN


In [128]:
f218

,f218,0
userid,,
100000001023,0 days 00:00:00,0.000000e+00
100000001505,1 days 04:46:17.750000,1.035778e+05
100000003461,0 days 00:00:00,0.000000e+00
100000005007,0 days 00:00:00,0.000000e+00
100000008119,0 days 00:00:00,0.000000e+00
100000009017,30 days 01:18:12.500000,2.596692e+06
100000009773,0 days 00:00:00,0.000000e+00
100000009925,0 days 00:41:10.500000,2.470500e+03
100000010199,27 days 11:07:17.625000,2.372838e+06
